Data Preprocessing notebook - Load OSM data generators wind power plants and solar - pv systems from geofabrik for complete countries and plot them

In [31]:
!pip install geopandas shapely esy-osmfilter plotly

In [32]:
import os
import requests
import shutil
#import warnings
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from esy.osmfilter import run_filter, Node, Way, Relation

In [33]:
def download_geofabrik_data(country_code, country_name, force_download=False):
    geofabrik_filename = f'{country_name}-latest.osm.pbf'
    geofabrik_url = f'https://download.geofabrik.de/africa/{geofabrik_filename}'
    PBF_inputfile = os.path.join(os.getcwd(), "pbf", geofabrik_filename)

    if not os.path.exists(PBF_inputfile) or force_download:
        print(f"{geofabrik_filename} does not exist, downloading to {PBF_inputfile}")
        os.makedirs(os.path.dirname(PBF_inputfile), exist_ok=True)
        with requests.get(geofabrik_url, stream=True) as r:
            with open(PBF_inputfile, 'wb') as f:
                shutil.copyfileobj(r.raw, f)

    return PBF_inputfile

def load_osm_data(PBF_inputfile, JSON_outputfile, prefilter, whitefilter, blackfilter,
                  NewPreFilterData, CreateElements, LoadElements, verbose=True):
    if not os.path.exists(JSON_outputfile):
        element_file_exists = False
    else:
        element_file_exists = True

    if not (Update or element_file_exists):
        create_elements = False
        new_prefilter_data = False
        print("Loading Pickle")
    else:
        create_elements = True
        new_prefilter_data = True
        print("Creating New Elements")

    [Data, Elements] = run_filter(elementname, PBF_inputfile, JSON_outputfile, prefilter,
                                  whitefilter, blackfilter, NewPreFilterData=new_prefilter_data,
                                  CreateElements=create_elements,
                                  LoadElements=LoadElements, verbose=verbose,
                                  multiprocess=True)

    return Data, Elements

def form_output(df_node):
    df_node['tags.generator:output:electricity'] = df_node['tags.generator:output:electricity'].str.replace(' kW', '').str.replace(' MW', '000')
    df_node['tags.generator:output:electricity'] = df_node['tags.generator:output:electricity'].str.replace(r'\D', '', regex=True).fillna('').replace('', np.nan).astype(float)
    return df_node

def process_relations_node(df_relation, Data):

    relations_node = []

    for index, row in df_relation.iterrows():
        for element in row['members']:
            if element[1] == 'NODE':
                extracts = pd.Series([element[0], element[1], element[2]], index=['osm_id', 'tag', 'gen'])
                new_row = pd.concat([row, extracts])
                relations_node.append(new_row)

    relations_node = pd.DataFrame(relations_node)

    if not relations_node.empty:
        relations_node['count'] = relations_node.groupby('id')['id'].transform('count')
        relations_node = form_output(relations_node)

    return relations_node

def process_relations_way(df_relation, Data):

    relations_way = []
    for index, row in df_relation.iterrows():
        for element in row['members']:
            if element[1] == 'WAY':
                extracts = pd.Series([element[0],element[1],element[2]],index=['osm_id','tag','gen'])
                row_with_extracts = pd.concat([row, extracts])
                relations_way.append(row_with_extracts)

    relations_way = pd.DataFrame(relations_way)

    if not relations_way.empty:
        relations_way['count'] = relations_way.groupby('id')['id'].transform('count')
        relations_way['tags.plant:output:electricity'] = relations_way['tags.plant:output:electricity'].str.replace('[^0-9.]','')
        relations_way['tags.plant:output:electricity'] = pd.to_numeric(relations_way['tags.plant:output:electricity'], errors='coerce')
        relations_way['tags.generator:output:electricity'] = relations_way['tags.plant:output:electricity'] / relations_way['count']

    return relations_way

def add_relation_to_node(df_node, relations_node):

    if not relations_node.empty:

       # relations_node['tags.generator:output:electricity'] = relations_node['tags.generator:output:electricity'].str.replace('[^0-9.]','')
        relations_node['tags.generator:output:electricity'] = pd.to_numeric(relations_node['tags.generator:output:electricity'], errors='coerce')
        relations_node['tags.generator:output:electricity'] = relations_node.apply(lambda row: row['tags.generator:output:electricity'] / 1000000 if row['tags.generator:output:electricity'] > 100000  else row['tags.generator:output:electricity'], axis=1)

        rel = relations_node[['osm_id','tags.generator:output:electricity', 'tags.name']]
        rel = rel.rename(columns={'osm_id': 'id'})
        rel = rel.set_index('id')
        df_node = df_node.set_index('id')
        df_node = df_node.fillna(rel).reset_index()

    else:
        pass

    return (df_node)

def ensure_columns(df):
    # List of columns you want in the final DataFrame
    columns_to_ensure = [
        'id',
        'lonlat',
        'tags.generator:method',
        'tags.generator:source',
        'tags.generator:output:electricity',
        'tags.generator:type',
        'tags.power',
        'tags.name'
    ]

    # Add missing columns with NaN values
    for column in columns_to_ensure:
        if column not in df.columns:
            df[column] = None  # You can use np.nan instead of None if you prefer

    # Reorder columns to match the desired order
    df = df[columns_to_ensure]

    return df

def calculate_area(geometry):
    # Calculate area for polygons, set NaN for points
    if isinstance(geometry, Polygon):
        return geometry.area * 10000000000
    else:
        return np.nan

In [34]:
country_mapping = {
    "DZ": {"country_name": "algeria", "country_code_long": "DZA"}#,
#    "EG": {"country_name": "egypt", "country_code_long": "EGY"},
#    "ZA": {"country_name": "south-africa", "country_code_long": "ZAF"}#,
  #         "AO": {"country_name": "angola", "country_code_long": "AGO"},
  #         "BJ": {"country_name": "benin", "country_code_long": "BEN"},
  #          "BW": {"country_name": "botswana", "country_code_long": "BWA"},
  #          "BF": {"country_name": "burkina-faso", "country_code_long": "BFA"},
  #          "BI": {"country_name": "burundi", "country_code_long": "BDI"},
  #          "CM": {"country_name": "cameroon", "country_code_long": "CMR"},
  #        ##   "IC": {"country_name": "canary-islands", "country_code_long": "CMR"}, ## funktioniert das node rechnen für punkte nicht!!
  #         "CV": {"country_name": "cape-verde", "country_code_long": "CPV"},
  #         "CF": {"country_name": "central-african-republic", "country_code_long": "CAF"},
  #         "TD": {"country_name": "chad", "country_code_long": "TCD"},
  #         ## "KM": {"country_name": "comoros", "country_code_long": "COM"}, ## kaputt DecodeError: Error parsing message with type 'OSMPBF.BlobHeader'
  #         "CG": {"country_name": "congo-brazzaville", "country_code_long": "COG"},
  #         "CD": {"country_name": "congo-democratic-republic", "country_code_long": "COD"},
  #         "DJ": {"country_name": "djibouti", "country_code_long": "DJI"},
  #         "GQ": {"country_name": "equatorial-guinea", "country_code_long": "GNQ"},
  #         "ER": {"country_name": "eritrea", "country_code_long": "ERI"},
  #         "ET": {"country_name": "ethiopia", "country_code_long": "ETH"},
  #         "GA": {"country_name": "gabon", "country_code_long": "GAB"},
  # #        "GM": {"country_name": "gambia", "country_code_long": "GMB"}, ## part of senegal
  #         "GH": {"country_name": "ghana", "country_code_long": "GHA"},
  #         "GN": {"country_name": "guinea", "country_code_long": "GIN"},
  #         "GW": {"country_name": "guinea-bissau", "country_code_long": "GNB"},
  #         "CI": {"country_name": "ivory-coast", "country_code_long": "CIV"},
  #         "KE": {"country_name": "kenya", "country_code_long": "KEN"},
  #         "LS": {"country_name": "lesotho", "country_code_long": "LSO"},
  #         "LR": {"country_name": "liberia", "country_code_long": "LBR"},
  #         "LY": {"country_name": "libya", "country_code_long": "LBY"},
  #         "MG": {"country_name": "madagascar", "country_code_long": "MDG"},
  #         "MW": {"country_name": "malawi", "country_code_long": "MWI"},
  #         "ML": {"country_name": "mali", "country_code_long": "MLI"},
  #         "MR": {"country_name": "mauritania", "country_code_long": "MRT"},
  #         "MU": {"country_name": "mauritius", "country_code_long": "MUS"},
  #         "MA": {"country_name": "morocco", "country_code_long": "MAR"},
  #         "MZ": {"country_name": "mozambique", "country_code_long": "MOZ"},
  #         "NA": {"country_name": "namibia", "country_code_long": "NAM"},
  #         "NE": {"country_name": "niger", "country_code_long": "NER"},
  #         "NG": {"country_name": "nigeria", "country_code_long": "NGA"},
  #         "RW": {"country_name": "rwanda", "country_code_long": "RWA"},
  #         "SH": {"country_name": "saint-helena-ascension-and-tristan-da-cunha", "country_code_long": "SHN"},
  #         "ST": {"country_name": "sao-tome-and-principe", "country_code_long": "STP"},
  #         "SN": {"country_name": "senegal-and-gambia", "country_code_long": "SEN"},
  #         "SC": {"country_name": "seychelles", "country_code_long": "SYC"},
  #         "SL": {"country_name": "sierra-leone", "country_code_long": "SLE"},
  #         "SO": {"country_name": "somalia", "country_code_long": "SOM"},
  #         "SS": {"country_name": "south-sudan", "country_code_long": "SSD"},
  #         "SD": {"country_name": "sudan", "country_code_long": "SDN"},
  #        ##  "SZ": {"country_name": "swasiland", "country_code_long": "SWZ"}, ##DecodeError: Error parsing message with type 'OSMPBF.BlobHeader'
  #         "TZ": {"country_name": "tanzania", "country_code_long": "TZA"},
  #         "TG": {"country_name": "togo", "country_code_long": "TGO"},
  #         "TN": {"country_name": "tunisia", "country_code_long": "TUN"},
  #         "UG": {"country_name": "uganda", "country_code_long": "UGA"},
  #         "ZM": {"country_name": "zambia", "country_code_long": "ZMB"},
  #         "ZW": {"country_name": "zimbabwe", "country_code_long": "ZWE"}
  # #         # Add more mappings as needed
    }

for country_code, info in country_mapping.items():

  print(f"Country Code: {country_code}")

  country_name = info['country_name']

  country_code_long = info["country_code_long"]

  Update = True
  JSON_outputfile = os.path.join(os.getcwd(), f'{country_name}_generator.json')
  elementname = f'{country_code}_substations'

  # Download Geofabrik data
  PBF_inputfile = download_geofabrik_data(country_code, country_name, force_download = Update)

  prefilter = {Node: {"power":["generator"]},
                Way: {"power":["generator"]},
                Relation: {"power":["generator","plant"]}}

  blackfilter = [("pipeline","generator"),
                  ("generator:source","gas"),
                  ("generator:source","nuclear"),
                  ("tags.generator:source", "hydro"),
                  ("source", "oil"),
                  ("source", "hydro"),
                  ("source","nuclear"),
                  ("generator:method", "combustion"),
                  ("generator:method", "run-of-the-river"),
                  ("generator:method", "stream"),
                  ("generator:method", "fission"),
                  ("generator:source", "hydro"),
                  ("generator:source", "gasoline"),
                  ("generator:source", "coal")]

  whitefilter =[[("power","generator"),],]

  # Load OSM data
  Data, Elements = load_osm_data(PBF_inputfile,
                                  JSON_outputfile,
                                  prefilter,
                                  whitefilter,
                                  blackfilter,
                                  NewPreFilterData=True,
                                  CreateElements=True,
                                  LoadElements=True)

  # =============================================================================
  #         # Process df_node
  # =============================================================================
  df_node = pd.json_normalize(Elements[elementname]["Node"].values())
  df_node.drop(columns=["tags.fixme", "tags.frequency"], inplace=True, errors='ignore')
  df_node = ensure_columns(df_node)
  df_node = form_output(df_node)

  # =============================================================================
  #         # Process relations_node
  # =============================================================================
  df_relation = pd.json_normalize(Data["Relation"].values())

  old_column_name = 'tags.plant:output:electricity'
  new_column_name = 'tags.generator:output:electricity'

  if old_column_name in df_relation.columns and new_column_name not in df_relation.columns:
      print('T')
      # Rename the column
      df_relation.rename(columns={old_column_name: new_column_name}, inplace=True)

  relations_node = process_relations_node(df_relation, Data)

  # =============================================================================
  #         add processed relations to nodes
  # =============================================================================
  df_node = add_relation_to_node(df_node, relations_node)

  export_node = df_node[['id', 'lonlat', 'tags.generator:method', 'tags.generator:source',
                    'tags.generator:output:electricity', 'tags.generator:type', 'tags.power',
                    'tags.name']]

  # =============================================================================
  # form dataframes with geoms , etc.
  # =============================================================================
  geom =[]

  for ref in export_node['lonlat']:

      geom.append(gpd.GeoSeries([Point(ref)])[0])

  export_node.drop(columns=['lonlat'], inplace=True, errors='ignore')

  export_node.insert(1, "lonlat", geom)

  node = gpd.GeoDataFrame(export_node, geometry='lonlat')

 # GeoJSON_windpower = os.path.join(path, f'{country_name}_windpowerplants.geojson')

Country Code: DZ
algeria-latest.osm.pbf does not exist, downloading to /content/pbf/algeria-latest.osm.pbf


INFO:esy.osmfilter.pre_filter:PreFilter OSM GAS DATA
INFO:esy.osmfilter.pre_filter:InputFile     : /content/pbf/algeria-latest.osm.pbf
INFO:esy.osmfilter.pre_filter:Size          : 281949          kbyte
INFO:esy.osmfilter.pre_filter:Estimated Time: 40.28           s
INFO:esy.osmfilter.pre_filter:=============================


Creating New Elements


INFO:esy.osmfilter.pre_filter:0.5
INFO:esy.osmfilter.pre_filter:1
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilt

T


In [35]:
node.head()

,id,lonlat,tags.generator:method,tags.generator:source,tags.generator:output:electricity,tags.generator:type,tags.power,tags.name
0,7869440109,POINT (8.63884 28.63162),NaN,solar,NaN,NaN,generator,NaN
1,7879954323,POINT (9.01328 28.67757),NaN,solar,NaN,NaN,generator,NaN
2,3469603544,POINT (2.97144 36.74764),wind_turbine,wind,NaN,horizontal_axis,generator,NaN
3,2178843696,POINT (3.52098 34.17435),NaN,NaN,NaN,NaN,generator,مصنع الباربا
4,7879954322,POINT (9.01331 28.67757),NaN,solar,NaN,NaN,generator,NaN


In [36]:
df_way = pd.json_normalize(Elements[elementname]["Way"].values(), errors='ignore')

geom = []

if not df_way.empty:
    for ref in df_way['refs']:
        lonlats = []
        for r in ref:
            lonlat = Data["Node"][str(r)]["lonlat"]
            lonlats.append(lonlat)

        try:
            geom.append(gpd.GeoSeries([Polygon(lonlats)])[0])

        except:
            geom.append(gpd.GeoSeries([LineString(lonlats)])[0])

    df_way.insert(1, "lonlat", geom)

df_way = ensure_columns(df_way)

df_way['area_m2'] = df_way['lonlat'].apply(calculate_area)

df_way['tags.generator:output:electricity'] = df_way['tags.generator:output:electricity'].str.replace('[^0-9.]','')
df_way['tags.generator:output:electricity'] = pd.to_numeric(df_way['tags.generator:output:electricity'], errors='coerce')

# Export df_way to GeoJSON
export_way = df_way[['id', 'lonlat', 'tags.generator:method', 'tags.generator:source',
                      'tags.generator:output:electricity', 'tags.generator:type', 'tags.power',
                      'tags.name', 'area_m2']]

geom = []

for ref in export_way['lonlat']:
    geom.append(gpd.GeoSeries([Polygon(ref[0])])[0] if isinstance(ref, list) else gpd.GeoSeries([ref])[0])

export_way.drop(columns=['lonlat'], inplace=True, errors='ignore')
export_way.insert(1, "lonlat", geom)

way = gpd.GeoDataFrame(export_way, geometry='lonlat')

<ipython-input-36-c35b7d394252>:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-c35b7d394252>:24: FutureWarning:

The default value of regex will change from True to False in a future version.

<ipython-input-36-c35b7d394252>:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-36-c35b7d394252>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [37]:
way.head()

,id,lonlat,tags.generator:method,tags.generator:source,tags.generator:output:electricity,tags.generator:type,tags.power,tags.name,area_m2
0,528166092,"POLYGON ((4.20072 34.86419, 4.20112 34.86419, ...",photovoltaic,solar,NaN,solar_photovoltaic_panel,generator,NaN,97.88215
1,380261513,"POLYGON ((3.36149 33.12217, 3.36150 33.12349, ...",thermal,solar,30.0,solar_thermal_collector,generator,NaN,566.45245
2,380257045,"POLYGON ((3.35297 33.12420, 3.35298 33.12552, ...",thermal,solar,30.0,solar_thermal_collector,generator,NaN,566.45245
3,527875153,"POLYGON ((3.16446 34.34773, 3.16446 34.34776, ...",photovoltaic,solar,NaN,solar_photovoltaic_panel,generator,NaN,113.74770
4,380262031,"POLYGON ((3.35844 33.12082, 3.35845 33.12214, ...",thermal,solar,30.0,solar_thermal_collector,generator,NaN,566.45245


In [44]:
import geopandas as gpd
import plotly.graph_objects as go

# Plot the points
fig = go.Figure(go.Scattermapbox(
    lat=node.geometry.y,
    lon=node.geometry.x,
    hovertext=node['tags.name'],
    mode='markers'
))

# Add the 'way' layer
fig.add_trace(go.Choroplethmapbox(
    geojson=way.geometry.__geo_interface__,
    locations=way.index,
    colorscale='Reds',
    z=[1] * len(way),  # Set a dummy value for coloring the polygons
    showscale=False
))

fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=2,
    mapbox_center={"lat": 0, "lon": 25},
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()